In [ ]:
import time

import json
import wandb
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import font_manager

from PIL import Image
import numpy as np
from easydict import EasyDict as edict

In [ ]:
for font in font_manager.fontManager.ttflist:
    if 'Nanum' in font.name:
        print(font.name, font.fname)

In [ ]:
import matplotlib as mpl

# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False

# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'NanumGothic'

In [ ]:
train_path = "/opt/ml/input/data/ICDAR17_Korean/ufo/train.json"
with open(train_path, "r") as f:
    train_json = json.load(f)
print(train_json.keys())
tr_dict = edict(train_json['images'])

In [ ]:
def isEngOrKor(str):
    lst = set()
    for c in str:
        if ord('가') <= ord(c) <= ord('힣'):
            lst.add('ko')
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            lst.add('en')
        else:
            continue
    return list(lst)

In [ ]:
tr_image_prefix = "/opt/ml/input/data/ICDAR17_Korean/images/"
rows = 1
cols = 2
for index, (key, value) in enumerate(tr_dict.items()):
    if index>=360 and index <400:
        fig = plt.figure(figsize=(20, 20)) # rows*cols 행렬의 i번째 subplot 생성
        ax1 = fig.add_subplot(rows, cols, 1)
        ax2 = fig.add_subplot(rows, cols, 2)
        im_name = tr_image_prefix + key
        Img = Image.open(im_name)
        ax1.imshow(Img)
        ax2.imshow(Img)
        for w_key, w_value in value['words'].items():
            if w_value['illegibility'] == False:
                print('orientation:', w_value['orientation'], end= ' / ')
                if isEngOrKor(w_value['transcription']) == w_value['language']:
                    print('compatibility:', isEngOrKor(w_value['transcription']) == w_value['language'], end= ' / ')
                else:
                    print('Algorithm:',isEngOrKor(w_value['transcription']), end=' / ')
                print('language:', w_value['language'], end= ' / ')
                print('transcription:', w_value['transcription'])
            x1, y1 = w_value['points'][0]
            points = w_value['points']
            tuple_points = [tuple(point) for point in points]
            tuple_points = tuple(tuple_points)
            rect = patches.Polygon(
                tuple_points,
                linewidth=3, edgecolor='deeppink', facecolor='none',
            )
            ax2.add_patch(rect)
            ax2.text(
            x=x1, y=y1, s=w_value['transcription'],
            bbox=dict(facecolor='deeppink', alpha=1)
            )

        plt.show()
